In [1]:
!pip3 install pyspark

Looking in indexes: https://dtn.jfrog.io/artifactory/api/pypi/pypi/simple

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [3]:
conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[6]')
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)
sc.setLogLevel("WARN")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/03 11:54:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/homebrew/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
sc

<SparkContext master=local[6] appName=Spark DL Tabular Pipeline>

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DecimalType, BooleanType, MapType, ArrayType

schema = StructType([
    StructField("uniq_id", StringType()),
    StructField("crawl_timestamp", DateType()),
    StructField("product_url", StringType()),
    StructField("product_name", StringType()),
    StructField("product_category_tree",StringType() ),
    StructField("pid", StringType()),
    StructField("retail_price", DecimalType(precision=38, scale=2)),
    StructField("discounted_price", DecimalType(precision=38, scale=2)),
    StructField("image", StringType()),
    StructField("is_FK_Advantage_product", BooleanType()),
    StructField("description", StringType()),
    StructField("product_rating", StringType()),
    StructField("overall_rating", StringType()),
    StructField("brand", StringType()),
    StructField("product_specifications", StringType())
])

product_category_tree = ArrayType(StringType())
image_schema = ArrayType(StringType())
product_specs_schema = MapType(keyType=StringType(), valueType=StringType())


In [6]:
source_file = "data/sample.csv"
data = sql_context.read.csv(source_file, schema=schema, inferSchema = True, header = True, escape='"').cache()


In [7]:
data.printSchema()


root
 |-- uniq_id: string (nullable = true)
 |-- crawl_timestamp: date (nullable = true)
 |-- product_url: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category_tree: string (nullable = true)
 |-- pid: string (nullable = true)
 |-- retail_price: decimal(38,2) (nullable = true)
 |-- discounted_price: decimal(38,2) (nullable = true)
 |-- image: string (nullable = true)
 |-- is_FK_Advantage_product: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- product_rating: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- product_specifications: string (nullable = true)



In [8]:
data.limit(5).toPandas()


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.00,379.00,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.00,22646.00,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.00,499.00,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.00,267.00,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.00,210.00,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [9]:
data.select("product_specifications").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
data.select("description").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [11]:
data.select("image").show(5, truncate=False)


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|image                                                                                                                                    

In [12]:
data.select("product_category_tree").show(5, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------+
|product_category_tree                                                                                                           |
+--------------------------------------------------------------------------------------------------------------------------------+
|["Clothing >> Women's Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women's Cycling Shorts"]|
|["Furniture >> Living Room Furniture >> Sofa Beds & Futons >> FabHomeDecor Fabric Double Sofa Bed (Finish Colo..."]             |
|["Footwear >> Women's Footwear >> Ballerinas >> AW Bellies"]                                                                    |
|["Clothing >> Women's Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women's Cycling Shorts"]|
|["Pet Supplies >> Grooming >> Skin & Coat Care >> Shampoo >> Sicons All Purpose Ar

## Transformation
We can see in our data that we have nested types, and clearly data scientists won't be able to query these properly. So, in order to have this done, we transform our data in a way to make things readable and queryable

In [13]:
from pydantic import BaseModel, validator
from typing import Optional, List

class Contents(BaseModel):
    key: str
    value: str

class ProductSpecs(BaseModel):
    product_specification: Optional[List[Contents]]

    @validator('product_specification', pre=True)
    def specs_check(cls, v):
        """
        pop out and ensure all dictionary going inside the function has the content attribute
        otherwise we discard them to reduce unwanted errors.
        """
        specs_attr = [i for i in Contents.__fields__.keys()]
        specs_attr_count = 0

        for i, x in enumerate(v):
            for k in dict(x).keys():
                if k in specs_attr:
                    specs_attr_count += 1
            if specs_attr_count != len(specs_attr):
                v.pop(i)
            specs_attr_count = 0
        return v

In [14]:
import numpy as np
import ast


def tree_mapper(categories: str):
    """
    Maps and strips the productname from the category tree

    :param categories: string of category tree the product has
    """
    return [category.strip() for category in categories.split(">>")][:-1]

def format_arr(arr: list):
    """
    Flatten and map items under category tree to get a list of categories

    :param arr: array of category tree
    """
    items = np.array([tree_mapper(item) for item in arr][:-1])
    flat =items.flatten()
    return flat.tolist()

def key_val_extractor(iterable: dict):
    if "key" in iterable:
        return {iterable["key"]: iterable["value"]}
    return {}

def format_str_to_dict(item: str):
    return ast.literal_eval(item.replace("=>", ":"))

def apply_mapping_attribute(item: str):
    """
    validate and flatten attributes for product specifics

    :param iterable: dictionary of product_specification
    """
    items = format_str_to_dict(item)
    mapping = {}
    if "product_specification" in items.keys():
        #validated_items = ProductSpecs(**item)
        for item in items["product_specification"]:
                mapping.update(key_val_extractor(item))

    return mapping




### Apply mini transformations

We just need to format the data, there should be no multiple aggregations done here since we want to follow an EtLT process.

In [15]:
from pyspark.sql.functions import col, udf


product_category_tree_udf = udf(lambda x:format_arr(ast.literal_eval(x)),product_category_tree) 
image_udf = udf(lambda x: ast.literal_eval(x), image_schema)
product_specifications_udf = udf(lambda x:apply_mapping_attribute(x),MapType(StringType(), StringType())) 
print(type(data))

<class 'pyspark.sql.dataframe.DataFrame'>


In [16]:
data = data.withColumn("image", image_udf(col("image")))
print(type(data))
data = data.withColumn("product_category_tree", product_category_tree_udf(col("product_category_tree")))
print(type(data))
data = data.withColumn("product_specifications", product_specifications_udf(col("product_specifications")))
print(type(data))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [17]:
data.printSchema()


root
 |-- uniq_id: string (nullable = true)
 |-- crawl_timestamp: date (nullable = true)
 |-- product_url: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_category_tree: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pid: string (nullable = true)
 |-- retail_price: decimal(38,2) (nullable = true)
 |-- discounted_price: decimal(38,2) (nullable = true)
 |-- image: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_FK_Advantage_product: boolean (nullable = true)
 |-- description: string (nullable = true)
 |-- product_rating: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- product_specifications: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [18]:
data.limit(5).toPandas()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,[],SRTEH2FF9KEDEFGF,999.00,379.00,[http://img5a.flixcart.com/image/short/u/4/a/a...,False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{'Ideal For': 'Women's', 'Style Code': 'ALTHT_..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,[],SBEEH3QGU7MFYJFY,32157.00,22646.00,[http://img6a.flixcart.com/image/sofa-bed/j/f/...,False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{'Shape': 'Square', 'Upholstery Color': 'Leath..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,[],SHOEH4GRSUBJGZXE,999.00,499.00,[http://img5a.flixcart.com/image/shoe/7/z/z/re...,False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{'Outer Material': 'Patent Leather', 'Ideal Fo..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,[],SRTEH2F6HUZMQ6SJ,699.00,267.00,[http://img5a.flixcart.com/image/short/6/2/h/a...,False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{'Ideal For': 'Women's', 'Style Code': 'ALTGHT..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,[],PSOEH3ZYDMSYARJ5,220.00,210.00,[http://img5a.flixcart.com/image/pet-shampoo/r...,False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{'Brand': 'Sicons', 'Form Factor': 'Liquid', '..."


In [19]:
data.select("product_specifications").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------